In [75]:
import urllib
from urllib import request
from urllib.parse import quote
from urllib.error import HTTPError
import socket
from urllib.error import URLError
from bs4 import BeautifulSoup
import csv
from urllib.request import *
import re
import string
import time

import numpy as np
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [142]:
def links(have_any=True):
    if have_any:
        old_link_df = pd.read_csv('news_links.csv')
    else:
        old_link_df = pd.DataFrame([], columns=['headlines','texts','rubrics','links'])
    return old_link_df

In [143]:
df = links(have_any=False)
df

,headlines,texts,rubrics,links


In [147]:
df

,headlines,texts,rubrics,links
0,Трамп назвал главу Нью-Йорка худшим мэром в США,Президент США Дональд Трамп назвал главу Нью-Й...,world,https://news.yandex.ru//story/Tramp_nazval_gla...
1,В Германии недовольные «Северным потоком — 2» ...,Протестующие против строительства «Северного п...,world,https://news.yandex.ru//story/V_Germanii_nedov...
2,США внесут Huawei в черный список,"Американский Минторг объявил, что внесёт крупн...",world,https://news.yandex.ru//story/SSHA_vnesut_Huaw...
3,Мадуро прокомментировал решение США запретить ...,"Президент Венесуэлы Николас Мадуро заявил, что...",world,https://news.yandex.ru//story/Maduro_prokommen...
4,Под завалами рухнувшего в Шанхае здания погибл...,Спасателям удалось обнаружить и вытащить из по...,world,https://news.yandex.ru//story/Pod_zavalami_ruk...
5,Появились тизерные изображения новой BMW 1-Series,Новый BMW 1-Series разработан на базе платформ...,world,https://news.yandex.ru//story/Poyavilis_tizern...
6,Philips перешла в Нидерландах на возобновляему...,Нидерландские предприятия Royal Philips NV пол...,world,https://news.yandex.ru//story/Philips_pereshla...
7,Виновницей крупнейшего в Калифорнии пожара наз...,Представители калифорнийского управления лесно...,world,https://news.yandex.ru//story/Vinovnicej_krupn...
8,Рождаемость в США достигла минимума,"По данным экспертов, снижение рождаемости набл...",world,https://news.yandex.ru//story/Rozhdaemost_v_SS...
9,В Турции анализ ДНК выявил новую тайну древней...,Международная группа генетиков и археологов из...,world,https://news.yandex.ru//story/V_Turcii_analiz_...


In [168]:
def crawling_links(start_link,rubric):
    
    def crawling_text(t_link):
        driver = webdriver.Chrome("C:\\Users\\Aleksandra\\Desktop\\chromedriver.exe")
        
        driver.get(t_link)
        driver.implicitly_wait(5)
        text = driver.find_element_by_class_name("doc__text").get_attribute('innerHTML')
        driver.quit()
        return text
        
    global df
    count = sum(1 for d in df['rubrics'] if d==rubric)
    
    if rubric == 'sport':
        start_link = 'https://yandex.ru/{0}'
        rub_link = start_link.format(rubric)
        
    else:
        rub_link = start_link.format(rubric)+'.html'    
        
    print(start_link.format(rubric)+'.html')
    
    url = request.urlopen(rub_link)
    soup = BeautifulSoup(url, 'html.parser')
    for i,l in enumerate(soup.find_all('a', {'class' : 'link_theme_black'})):
        
        if not i % 10: # каждые 10 операций смотрим, как там идут дела
            print(df['texts'][-10:])
            
        if not i%50: #сохраняю в файл каждые 50 итераций
            df.to_csv('texts.csv', encoding='utf-8')
        
        if count>14:
                return
            
        try:
            new_link=l.get('href')
            #удостоверились, что это не ссыль на рубрику
            #new_link = new_link[:new_link.find('?lr')]
            #print(new_link)
            
            if rubric == 'sport':
                new_link = new_link[1:] # там в начале лишний /
            
            if '/story/' in new_link and not new_link in list(df['links']):
                count+=1
                headline = l.string
                
                t_link = start_link.format(new_link)
                #print(t_link)
                
                text = crawling_text(t_link)
                #print(headline,text)
                if text:
                    df.loc[len(df)]=[headline,text,rubric,t_link]
                else:
                    continue
            
                
            
        except AttributeError:
            #print('__ НЕТ АТРИБУТА В : __', l)
            pass

        except KeyError:
            #print('__ НЕТ КЛЮЧА В : __', l)
            pass
    #print('page {0}, count {1}'.format(numb, count))'''
    #print(prod_link)

In [173]:
rubrics = {'world':'В мире','politics':'Политика','society':'Общество','business':'Экономика','sport':'Спорт','incident':'Происшествия','culture':'Культура','computers':'Технологии','science':'Наука'}
start_link = 'https://news.yandex.ru/{0}'

for r in rubrics:
    crawling_links(start_link,r)

https://news.yandex.ru/world.html
92     В суд на певицу подал кубинский композитор Лив...
93     Скульптуру под названием "Кролик", созданную а...
94     В отличие от ранее принятого (и уже отмененног...
95     Букмекеры считают, что победителем песенного к...
96     Главную награду международного Каннского киноф...
97     Кинопоиск опубликовал трейлер второго полномет...
98     На шоу мальчик представлял команду певицы Пела...
99     Добби/Гарри Поттер и&nbsp;Тайная комната Депре...
100    Грузинская певица Тамта, представляющая Кипр н...
101    Американская компания TBS продлила сериал «Чуд...
Name: texts, dtype: object
https://news.yandex.ru/politics.html
92     В суд на певицу подал кубинский композитор Лив...
93     Скульптуру под названием "Кролик", созданную а...
94     В отличие от ранее принятого (и уже отмененног...
95     Букмекеры считают, что победителем песенного к...
96     Главную награду международного Каннского киноф...
97     Кинопоиск опубликовал трейлер второго по

In [181]:
df.to_csv('texts1.csv',sep='\t', encoding = 'utf-8')